#Notebook
In this noteboook, use the exported tiff files to
-find min max (excl. <-30 dB values)
-calculate percentage of -30 dB pixels over images
-scale them
-remove/filter the -30 dB values
-write to tfrec



## Load libraries

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import numpy as np
import pathlib
import matplotlib.pyplot as plt
# import heapq, random
import pandas as pd

In [3]:
from sklearn.utils import shuffle



In [4]:
!pip install -q rasterio
import rasterio

     |████████████████████████████████| 19.3 MB 1.4 MB/s 


In [5]:
import tensorflow as tf

In [6]:
import random


In [51]:
import tqdm

# Defintions

## Load (definitions)

### load files

In [7]:
def subsplit(x):
  return x.split("_",1)[-1].split(".tif")[0]


def select_corresponding_files(files_x,files_y,files_dem):
  id_x = []
  for i in range(len(files_x)): #X becomes same as y, dif sequence than in X
    id_temp = subsplit(files_x[i])
    id_x.append(id_temp)

  id_y = [] #longer list possibly
  for i in range(len(files_y)): 
    id_temp = subsplit(files_y[i])
    id_y.append(id_temp)

  temp = set(id_x)
  res = [i for i, val in enumerate(id_y) if val in temp]

  files_y_corresponding = []
  for j in res: 
     files_y_corresponding.append(files_y[j])

  id_dem = []
  for k in range(len(files_dem)): 
    id_temp = subsplit(files_dem[k])
    id_dem.append(id_temp)

  temp = set(id_x)
  res = [k for k, val in enumerate(id_dem) if val in temp]

  files_dem_corresponding = []
  for l in res: 
     files_dem_corresponding.append(files_dem[l])

  return files_y_corresponding,files_dem_corresponding


In [38]:

def get_files_ls(x_path, y_path,dem_path):
  """"Make sure that the files x and y containing landslides are corresponding 
  """

  files_y = [str(f) for f in pathlib.Path(y_path).glob('*.tif')] #binary
  files_x = [str(f) for f in pathlib.Path(x_path).glob('*.tif')] #prepost
  files_dem = [str(f) for f in pathlib.Path(x_path).glob('*.tif')] #dem

  files_x = remove_duplicates_x(files_x)
  files_y = remove_duplicates_x(files_y)
  files_dem = remove_duplicates_x(files_dem)
  
  # len_y = len(files_y)
  # len_x = len(files_x)

  files_x.sort(key = lambda x: int(subsplit(x)))
  files_y.sort(key = lambda x: int(subsplit(x)))

  # if (len_x != len_y):
  files_y,files_dem = select_corresponding_files(files_x,files_y,files_dem)

  return files_x,files_y,files_dem



Have to add dem to x as one file before train,test,val split. Maybe just do addBands?

In [9]:
# def make_multiband(files_x,files_dem):
#   #not sure if addBands works once it is tiff file...
#   for i in range(len(files_x)):
#     image_x = files_x[i]
#     image_dem = files_dem[i]
#     image = image_x.addBands(image_dem)

#   #or after open raster..


### open files

In [10]:
# def open_tiff_x(x):
#   """"" Open tiff file x. The sar 1 and sar 2 band was accidentally wrongly turned with exporting
#   so that is now redone here. The output are the three bands of the x tiff image
#   """
#   raster = rasterio.open(x)
#   #accidentally the 1st band was sar2, second band sar1
#   sar = raster.read()
#   return sar


In [11]:

def open_tiff(y):
  raster = rasterio.open(y)
  y = raster.read()
  # shape needs to be (256,256,1) (currently 1,256,256) Might depend per region what they have..
  y_moved = np.moveaxis(y, 0, -1) #np array, source  (axis), destination (axis)
  return y_moved


In [12]:
# def open_tiff_dem(dem): #actually same as open_tiff_x
#   raster = rasterio.open(dem)
#   dem = raster.read()
#   return dem

### divide into train , test

In [13]:
from sklearn.model_selection import train_test_split

import pandas as pd

In [14]:
# def split_train_test_val(x,y,test_size,val_size):
#   """" Divide data into a train, val and test set 
#   Train is for training model, val is for validating (and thus choosing) model during training,
#   test is for testing the final model after training and thus qualifying the model
#   For a division of 0.7,0.1,0.2 then use test size = 0.2, and then test_size = 0.125
#   """
#   x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=test_size, random_state=1)
#   x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=val_size, random_state=1) # 0.125 x 0.8 = 0.1
#   return x_train, y_train,x_test,y_test,x_val,y_val

In [15]:
# def split_train_test(x,y,test_size):
#   """" Divide data into a train, test set 
#   Train is for training model,
#   test is for testing the final model after training and thus qualifying the model
#   """
#   x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=test_size, random_state=1)
#   return x_train, y_train,x_test,y_test

In [16]:
#make into df


def split_train_test_val(x,y,dem,test_size=0.2,val_size=0.1):
  #make df first, such that x,y and dem can be split
  df = pd.DataFrame({"x":x,"y":y,"dem":dem})
  #shuffle df

  #split into ttv
  seed = 42
  np.random.seed(seed)
  perm = np.random.permutation(df.index)
  m = len(df.index)

  train_percent = 0.7
  train_end = int(train_percent * m)
  validate_end = int(val_size * m) + train_end
  train = df.iloc[perm[:train_end]]
  validate = df.iloc[perm[train_end:validate_end]]
  test = df.iloc[perm[validate_end:]]
  return train, validate, test #these are df's .. 


## Get min, max

In [17]:
# Remove -30 dB and lower  pixel values
#Save these sar bands to new npz
def get_min_max(x):
  
  #combine with min max?
  sqGridSar = []

  for i in range(len(x)):
    raster = rasterio.open(x[i])
    sar = raster.read()
    sqGridSar.append(sar)
 

  #Get min max
  max_sar_arr = np.nanmax(sqGridSar,axis = None)
  min_sar_arr = np.nanmin(sqGridSar,axis = None) #this should be -29!


  return min_sar_arr,max_sar_arr


  # Remove -30 dB and lower  pixel values
#Save these sar bands to new npz
def get_max(x):
  
  #combine with min max?
  sqGridSar = []

  for i in range(len(x)):
    raster = rasterio.open(x[i])
    sar = raster.read()
    sqGridSar.append(sar)
 

  #Get min max
  max_sar_arr = np.nanmax(sqGridSar,axis = None)

  return max_sar_arr

## scale values def

the min sar value can be put to -1, since this will always be -29 due to recommendation to filter out <-30
The -30 dB are also scaled to -1. 
The max sar value can be scaled to 0.5 or 0.8 or something like it

The max and min dem still need -0.8,0.8 or something WITHIN range of -1,1

The target y can be -1,1 obviously. Not sure if that is bad to have the target be -1,1 but the source -0.8,0.8 and another source band -1,0.8 :S

For only ls
des max sar is 26 and max dem is 85. The max sar comes from hiro, max dem comes from hu
asc max sar is 24 and max dem is 85. The max sar comes from hiro,max dem comes from hu

In [18]:
#function to map from left (original) to right (scaled)
def scaleGrid(input_grid,leftMin,leftMax, rightMin,rightMax):
  '''
  objective: map the values of the input grid to the proposed range
  input_grid: grid with values
  leftMin,leftMax: min and max of the original input grid
  rightMin,rightMax: min and max of the range to which the data is mapped  
  '''
 
  leftSpan = leftMax - leftMin
  rightSpan= rightMax - rightMin
 
  valueScaled = (input_grid - leftMin) / float(leftSpan)
  valueScaled = np.round(rightMin + (valueScaled * rightSpan),4)

 
  return valueScaled

In [19]:
def remove_nan(arr,fill):
  arr[np.isnan(arr)] = fill
  return arr

In [60]:
def scale_x(sar,min_sar,max_sar,max_value = 0.8):

  #scale X 
  sar = scaleGrid(sar,min_sar,max_sar,-0.8,max_value)
  sar = remove_nan(sar,fill = -1)
  return sar

In [59]:
def scale_dem(dem,max_value = 0.8):
  
  #remove <0 dem values, change to 0 values
  #how...
  dem[dem<0] = 0


  #scale dem
  min_dem = 0
  max_dem = 85 
  dem = scaleGrid(dem,min_dem,max_dem,-1,max_value)
  dem = remove_nan(dem,fill = -1)
  return dem

In [22]:
def scale_y(y):
  fill = -1
  scaled_y = scaleGrid(y,0,255,-1,1)
  scaled_y = remove_nan(scaled_y,fill)
  return scaled_y


In [23]:
def to_np_float(arr):
  arr = np.float32(arr)
  return arr

In [24]:
def get_tile_nr(x):
  return x.split("_",1)[-1].split(".tif")[0] 

def remove_duplicates(files_x,files_y): #does this work well... Should be other way around to..
  for item in files_x:
    tile_nr = get_tile_nr(item)
    if ("(" in item):
      # print(item)
      files_x.remove(item)
  for item in files_y:
    tile_nr = get_tile_nr(item)
    if ("(" in item):
      files_y.remove(item)

  return files_x,files_y

## Convert to tfRecord

### Serialize example

In [25]:
def _bytes_feature(value):
  """Returns a bytes_list from a string / byte."""
  if isinstance(value, type(tf.constant(0))):
    value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
  """Returns a float_list from a float / double."""
  return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _int64_feature(value):
  """Returns an int64_list from a bool / enum / int / uint."""
  return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

# def _int32_feature(value):
#   """Returns an int64_list from a bool / enum / int / uint."""
#   return tf.train.Feature(int32_list=tf.train.Int32List(value=[value]))

# def serialize_array(array):
#   array = tf.io.serialize_tensor(array)
#   return array

In [26]:

def serialize_example(path_x,path_y,path_dem,min_sar,max_sar):

  #open y file
  y = open_tiff(path_y)
  y = scale_y(y)

  #open the x tiff file
  x = open_tiff(path_x)
  x = scale_x(x,min_sar,max_sar)

  dem = open_tiff(path_dem)
  dem = scale_dem(dem)

  
  #change all dtypes of array to float32, as float64 is not accepted by tfrec
  x = to_np_float(x)
  y = to_np_float(y)
  dem = to_np_float(dem)


  feature = {}
  feature['x'] = _bytes_feature(x.tobytes())
  feature['Y'] = _bytes_feature(y.tobytes())
  feature['dem'] = _bytes_feature(dem.tobytes())
    
  return tf.train.Example(features=tf.train.Features(feature=feature)).SerializeToString()




add metadata

### Write

In [52]:
from tqdm.notebook import tqdm

In [27]:
def writeTFRecord(path_name,path_x,path_y,path_dem,min_sar,max_sar,samples_per_tf):
  
  #path_x should be a [str(f) for f in path]
  # samples_per_tf = 100
  n_samples = len(path_x)


  splitNum = int(np.ceil(n_samples/samples_per_tf))

  for index in tqdm(range(splitNum)):
    partition_factor = np.round(n_samples/splitNum) #split number?
    path_x_part = path_x[int(partition_factor*index):int(partition_factor*(index+1))] #shape is 100,256,256
    path_y_part = path_y[int(partition_factor*index):int(partition_factor*(index+1))]
    path_dem_part = path_dem[int(partition_factor*index):int(partition_factor*(index+1))]

    record_file = path_name +"%d.tfrec" %index #index seems wrong, the name here at least!

    with tf.io.TFRecordWriter(record_file) as writer: 
      for i in tqdm(range(len(path_x_part))):
        writer.write(serialize_example(path_x_part[i],path_y_part[i],path_dem_part[i],min_sar,max_sar))


In [28]:
def run_write(f_x,f_y,f_dem,write_path,name_tfrec_file,min_sar,max_sar):
  """"" 
  f_x: tiff files of x images
  f_y : tiff files of y images
  write_path: path to local folder where tfrec shoudl be written to
  name_tfrec_files : name which has to end with _
  each tfrec file has a this name _ number, in such a way multiple tfrec files can be added to same kaggle dataset, otherwise names would be duplicate
  """""


 #if file already exists this gives error, just command this line then:
  # os.makedirs(write_path)
  # writeTFRecord(write_path+name_tfrec_file,f_x,f_y,100)
  writeTFRecord(write_path+name_tfrec_file,f_x,f_y,f_dem,min_sar,max_sar,100)
                

In [29]:
def mix_regions(x1,x2,x3,x4,y1,y2,y3,y4,dem1,dem2,dem3,dem4):
  x = x1 + x2 + x3 + x4
  y = y1 + y2+ y3 + y4
  dem = dem1+dem2+dem3+dem4
  c = list(zip(x,y,dem))
  random.shuffle(c)
  x,y,dem = zip(*c)
  return x,y,dem


In [30]:
def getlistfromdf(df):
  x = df.iloc[:,0]
  x = x.tolist()

  y = df.iloc[:,1]
  y = y.tolist()

  z = df.iloc[:,2]
  z = z.tolist()

  return x,y,z

In [31]:

def get_files(x_path,y_path,dem_path):
  #get files x and y
  fx,fy,fdem = get_files_ls(x_path, y_path,dem_path)
  #split intro train test val
  train,test,val = split_train_test_val(fx,fy,fdem)
  x_train,y_train,dem_train = getlistfromdf(train)
  x_test,y_test,dem_test = getlistfromdf(test)
  x_val,y_val,dem_val = getlistfromdf(val)


  x_sets = [x_train,x_test,x_val]
  y_sets = [y_train,y_test,y_val]
  dem_sets = [dem_train,dem_test,dem_val]
  #get (min) max of x_train
  min_x,max_x = get_min_max(x_train)
  # max_x = get_max(x_train)
  return min_x,max_x,x_sets,y_sets,dem_sets

In [40]:

def remove_duplicates_x(files_x):
  for item in files_x:
    tile_nr = get_tile_nr(item)
    if ("(" in item):
      # print(item)
      files_x.remove(item)
  return files_x

# Run functions

In [32]:
#files Y
path_y = "/content/drive/MyDrive/Thesis/Y/"
!ls -la {path_y}

total 16
drwx------ 2 root root 4096 Oct  7 13:18 Hiro
drwx------ 2 root root 4096 Oct  7 13:18 Hok
drwx------ 2 root root 4096 Oct  7 13:18 Hu
drwx------ 2 root root 4096 Oct  7 13:46 Lom


In [33]:
#files X
path_x_t = "/content/drive/MyDrive/Thesis/Dem/"
!ls -la {path_x_t}

path_x = path_x_t + "Aratio/"
path_dem = path_x_t + "Dem/"

total 8
drwx------ 2 root root 4096 Oct  7 13:20 Aratio
drwx------ 2 root root 4096 Oct  7 13:20 Dem


In [44]:
path_x1 = path_x + "Hu/"
path_x2 = path_x + "Hiro/"
path_x3 = path_x + "Hok/"
path_x4 = path_x + "Lombok/"

path_y1 = path_y + "Hu/"
path_y2 = path_y + "Hiro/"
path_y3 = path_y + "Hok/"
path_y4 = path_y + "Lom/"

path_dem1 = path_dem + "Hu/"
path_dem2 = path_dem + "Hiro/"
path_dem3 = path_dem + "Hok/"
path_dem4 = path_dem + "Lombok/"

In [35]:

#mix the various regions.
#x gives [trainx,testx,valx] likewise y and dem
min_x1,max_x1,x1,y1,dem1 = get_files(path_x1,path_y1,path_dem1) #works!


In [41]:
min_x2,max_x2,x2,y2,dem2 = get_files(path_x2,path_y2,path_dem2) 

In [42]:
min_x3,max_x3,x3,y3,dem3 = get_files(path_x3,path_y3,path_dem3)


In [45]:
min_x4,max_x4,x4,y4,dem4 = get_files(path_x4,path_y4,path_dem4)

In [48]:
print(max_x1,max_x2,max_x2,max_x4)
max_x = max([max_x1,max_x2,max_x2,max_x4])
print(max_x)

min_x = min([min_x1,min_x2,min_x3,min_x4])
print(min_x1,min_x2,min_x3,min_x4)
print(min_x)

train_x,train_y,train_dem = mix_regions(x1[0],x2[0],x3[0],x4[0],y1[0],y2[0],y3[0],y4[0],dem1[0],dem2[0],dem3[0],dem4[0])
val_x,val_y,val_dem = mix_regions(x1[1],x2[1],x3[1],x4[1],y1[1],y2[1],y3[1],y4[1],dem1[1],dem2[1],dem3[1],dem4[1])
test_x,test_y,test_dem = mix_regions(x1[2],x2[2],x3[2],x4[2],y1[2],y2[2],y3[2],y4[2],dem1[2],dem2[2],dem3[2],dem4[2])

12.22187711929773 12.783960937574514 12.783960937574514 8.17182893766089
12.783960937574514
-16.79680289037293 -15.64489334922045 -9.797669073662924 -10.930583071874153
-16.79680289037293


In [61]:
min_x

-16.79680289037293

In [62]:
tfrec_path = "/content/drive/MyDrive/Thesis/0710_tfrec_dif_dem_08/"
os.makedirs(tfrec_path)
tfrec_path_train = tfrec_path + "train/"
tfrec_path_test = tfrec_path + "test/"
tfrec_path_val = tfrec_path + "val/"
os.makedirs(tfrec_path_train)
os.makedirs(tfrec_path_test)
os.makedirs(tfrec_path_val)

In [63]:
#convert to tfrecord (scale x, scale y incl.)
run_write(train_x,train_y,train_dem,tfrec_path_train,'dif_train_dem_ls_',min_x,max_x)
# run_write(f_x = train_x,f_y = train_y, f_dem =

  0%|          | 0/13 [00:00<?, ?it/s]

  0%|          | 0/99 [00:00<?, ?it/s]

  0%|          | 0/99 [00:00<?, ?it/s]

  0%|          | 0/99 [00:00<?, ?it/s]

  0%|          | 0/99 [00:00<?, ?it/s]

...

  0%|          | 0/99 [00:00<?, ?it/s]

  0%|          | 0/99 [00:00<?, ?it/s]

  0%|          | 0/99 [00:00<?, ?it/s]

  0%|          | 0/99 [00:00<?, ?it/s]

  0%|          | 0/98 [00:00<?, ?it/s]

In [64]:
run_write(test_x,test_y,test_dem,tfrec_path_test,'dif_test_dem_ls',min_x,max_x) #why only two sooo annoying!


  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/93 [00:00<?, ?it/s]

  0%|          | 0/93 [00:00<?, ?it/s]

  0%|          | 0/93 [00:00<?, ?it/s]

  0%|          | 0/92 [00:00<?, ?it/s]

In [65]:
run_write(val_x,val_y,val_dem,tfrec_path_val,'dif_val_dem_ls',min_x,max_x)

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/91 [00:00<?, ?it/s]

  0%|          | 0/91 [00:00<?, ?it/s]